# Final Assignment B
Kamile Stankeviciute `ks830`

In [56]:
# Imports
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
# Loading the dataset
pd.set_option('display.max_columns', 500)

diabetic = pd.read_csv('diabetes/diabetic_data_original.csv')
diabetic.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [6]:
diabetic['encounter_id'].sort_values()

8             12522
9             15738
4             16680
10            28236
5             35754
            ...    
101761    443847548
101762    443847782
101763    443854148
101764    443857166
101765    443867222
Name: encounter_id, Length: 101766, dtype: int64

In [10]:
patient_nbr_counts = diabetic['patient_nbr'].value_counts()
df = diabetic[diabetic['patient_nbr'].isin(patient_nbr_counts.index[patient_nbr_counts.gt(1)])].sort_values(['patient_nbr', 'encounter_id'])
pd.set_option('display.max_rows', 25)
print(df[['encounter_id', 'patient_nbr', 'readmitted']])

        encounter_id  patient_nbr readmitted
4267        24437208          135        <30
4780        26264286          135        >30
1164         8380170         1152        >30
5953        30180318         1152        >30
14180       55533660         1152        >30
23623       80742510         1152        >30
24642       83281464         1152         NO
15848       60254142         1314        >30
19765       70190028         1314        <30
19914       70601076         1314         NO
1083         7981038         5220        >30
6901        33503946         5220        >30
...              ...          ...        ...
93582      337315430    187920311         NO
99386      412637672    187920311        <30
96251      372665912    188238020         NO
99239      410904182    188238020        <30
94175      344902604    188284883        >30
97639      390910244    188284883         NO
95640      364338650    188634893        >30
99544      414418886    188634893         NO
93050     

## Training task

*For the purposes of this assignment, we wish to predict the `time_in_hospital` value. It doesn’t make sense to predict the duration of a stay in hospital by using attributes recorded during that stay. Instead, we will restrict attention to **follow-on visits**, and seek to predict the length of a stay for follow-on visit on the basis of attributes recorded in the previous visit.*

*Follow-on* visits mean that the model will be trained on *patients who had follow-on visits*. I will select the patients who have been readmitted (only those `patient_nbr`s which appear more than once in the dataset), sort them by time (assuming the `encounter_id` is sorted and sequential), ignoring the patients who have not been readmitted. Then 
* the train instance will contain data from the first visit, 
* the test instance will contain length of stay (`time_in_hospital`) of the next visit


Q: What about the patients which had several follow-ons? Can the first follow-up (ground truth for the initial visit) be a training instance, with the second follow-up having the ground truth? Is there data leakage? A: No as long as just the time in hospital is given.

Q: Can I use the length of stay of the current visit as a feature? A: Yes, why not.

## Data preparation

*Don’t explain the code you used, but do explain what filtering or processing you applied, and report how many items there are in your dataset.*

* Patient with follow-on visits selection.
* Generating predicted variable (`next_time_in_hospital`) by extracting the `time_in_hospital` of the same `patient_nbr` from the next `encounter_id`.
* Scaling (normalisation): important for training stability.
* Categorial variable one-hot coding.
* Handling missing data: masks and codes.
* Feature engineering and selection should not be necessary with a deep learning framework, which, with sufficient model complexity, should learn to extract those features by itself.

**Q: Can I assume `encounter_id`s are chronological? A: assumption is reasonable as manually looking at the dataset with sorted encounters and patient identifiers all entries have readmitted outcome but the last outcome is of not readmitted.** It has since been updated that this is the correct assumption.

Processing can be described in the context of the last assignment.

## Preprocessing pipeline
Filtering to only include patients with more than one encounter:

In [123]:
patient_nbr_counts = diabetic['patient_nbr'].value_counts()
pd.set_option('display.max_rows', 20)
df = diabetic[diabetic['patient_nbr'].isin(patient_nbr_counts.index[patient_nbr_counts.gt(1)])].sort_values(['patient_nbr', 'encounter_id']).copy()
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
4267,24437208,135,Caucasian,Female,[50-60),?,2,1,1,8,?,Cardiology,77,6,33,0,0,0,401,997,560,8,None,None,Steady,No,No,No,No,No,No,Down,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,<30
4780,26264286,135,Caucasian,Female,[50-60),?,1,1,7,3,?,Surgery-Cardiovascular/Thoracic,31,1,14,0,0,1,998,41,250,5,None,None,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Ch,Yes,>30
1164,8380170,1152,AfricanAmerican,Female,[50-60),?,1,1,7,6,?,Hematology/Oncology,43,2,13,0,0,1,282,250.01,?,2,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
5953,30180318,1152,AfricanAmerican,Female,[50-60),?,1,1,7,6,?,Hematology/Oncology,45,4,15,0,0,2,282,794,250,6,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30
14180,55533660,1152,AfricanAmerican,Female,[60-70),?,1,1,7,10,?,Hematology/Oncology,54,2,19,0,0,1,282,276,428,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
23623,80742510,1152,AfricanAmerican,Female,[60-70),?,1,1,7,8,?,?,30,1,16,0,0,1,282,250,?,2,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
24642,83281464,1152,AfricanAmerican,Female,[60-70),?,1,1,7,12,?,Hematology/Oncology,37,1,18,0,0,2,282,287,466,5,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
15848,60254142,1314,Caucasian,Female,[40-50),?,2,1,1,2,?,InternalMedicine,50,5,13,0,0,0,996,411,401,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
19765,70190028,1314,Caucasian,Female,[40-50),?,1,1,7,1,?,InternalMedicine,47,5,12,0,0,1,414,411,250.01,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,<30
19914,70601076,1314,Caucasian,Female,[40-50),?,1,1,7,3,?,Cardiology,54,3,14,0,0,2,78,250,414,7,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
df[df['readmitted'] != 'NO']['patient_nbr'].value_counts()

Encoding of categorical, numerical features.

In [17]:
df['patient_nbr'].value_counts()

88785891     40
43140906     28
23199021     23
88227540     23
1660293      23
84428613     22
23643405     22
92709351     21
23398488     20
37096866     20
             ..
50363289      2
100479627     2
66457080      2
4893336       2
84415995      2
41271957      2
42743295      2
20454030      2
107217549     2
40980195      2
Name: patient_nbr, Length: 16773, dtype: int64

In [ ]:
df.info()

In [49]:
df['readmitted'].value_counts()

>30    25667
NO     12163
<30     9191
Name: readmitted, dtype: int64

## Marking unknown values

In [29]:
df['weight'].value_counts() / len(df)

?            0.986389
[75-100)     0.005551
[50-75)      0.004275
[100-125)    0.002510
[125-150)    0.000510
[25-50)      0.000340
[0-25)       0.000276
[175-200)    0.000085
[150-175)    0.000043
>200         0.000021
Name: weight, dtype: float64

In [32]:
df['medical_specialty'].value_counts() / len(df)

?                                   0.509538
InternalMedicine                    0.123838
Emergency/Trauma                    0.097977
Family/GeneralPractice              0.077412
Cardiology                          0.040110
Surgery-General                     0.028158
Nephrology                          0.023373
Orthopedics                         0.011080
Radiologist                         0.011038
Psychiatry                          0.008103
                                      ...   
Pediatrics-Hematology-Oncology      0.000043
Surgery-Pediatric                   0.000043
Surgery-PlasticwithinHeadandNeck    0.000021
Cardiology-Pediatric                0.000021
Dentistry                           0.000021
Pediatrics-InfectiousDiseases       0.000021
Pediatrics-EmergencyMedicine        0.000021
Dermatology                         0.000021
SportsMedicine                      0.000021
Resident                            0.000021
Name: medical_specialty, Length: 68, dtype: float64

In [124]:
for column in list(df.columns.values):
    df[column] = np.where(df[column] == '?', np.nan, df[column])
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
4267,24437208.0,135.0,Caucasian,Female,[50-60),NaN,2.0,1.0,1.0,8.0,NaN,Cardiology,77.0,6.0,33.0,0.0,0.0,0.0,401,997,560,8.0,None,None,Steady,No,No,No,No,No,No,Down,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,<30
4780,26264286.0,135.0,Caucasian,Female,[50-60),NaN,1.0,1.0,7.0,3.0,NaN,Surgery-Cardiovascular/Thoracic,31.0,1.0,14.0,0.0,0.0,1.0,998,41,250,5.0,None,None,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Ch,Yes,>30
1164,8380170.0,1152.0,AfricanAmerican,Female,[50-60),NaN,1.0,1.0,7.0,6.0,NaN,Hematology/Oncology,43.0,2.0,13.0,0.0,0.0,1.0,282,250.01,NaN,2.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
5953,30180318.0,1152.0,AfricanAmerican,Female,[50-60),NaN,1.0,1.0,7.0,6.0,NaN,Hematology/Oncology,45.0,4.0,15.0,0.0,0.0,2.0,282,794,250,6.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30
14180,55533660.0,1152.0,AfricanAmerican,Female,[60-70),NaN,1.0,1.0,7.0,10.0,NaN,Hematology/Oncology,54.0,2.0,19.0,0.0,0.0,1.0,282,276,428,9.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
23623,80742510.0,1152.0,AfricanAmerican,Female,[60-70),NaN,1.0,1.0,7.0,8.0,NaN,NaN,30.0,1.0,16.0,0.0,0.0,1.0,282,250,NaN,2.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
24642,83281464.0,1152.0,AfricanAmerican,Female,[60-70),NaN,1.0,1.0,7.0,12.0,NaN,Hematology/Oncology,37.0,1.0,18.0,0.0,0.0,2.0,282,287,466,5.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
15848,60254142.0,1314.0,Caucasian,Female,[40-50),NaN,2.0,1.0,1.0,2.0,NaN,InternalMedicine,50.0,5.0,13.0,0.0,0.0,0.0,996,411,401,9.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
19765,70190028.0,1314.0,Caucasian,Female,[40-50),NaN,1.0,1.0,7.0,1.0,NaN,InternalMedicine,47.0,5.0,12.0,0.0,0.0,1.0,414,411,250.01,7.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,<30
19914,70601076.0,1314.0,Caucasian,Female,[40-50),NaN,1.0,1.0,7.0,3.0,NaN,Cardiology,54.0,3.0,14.0,0.0,0.0,2.0,78,250,414,7.0,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


## Creating `length_of_next_stay` feature

In [125]:
df = df.set_index(df['encounter_id'])

In [78]:
encounter_ids = list(df[df['patient_nbr'] == 1314.0]['encounter_id'])
encounter_ids

[60254142.0, 70190028.0, 70601076.0]

In [80]:
lengths_of_stay = df.loc[encounter_ids]['time_in_hospital']
lengths_of_stay

encounter_id
60254142.0    2.0
70190028.0    1.0
70601076.0    3.0
Name: time_in_hospital, dtype: float64

In [95]:
lengths_of_next_stay = lengths_of_stay.values[1:]
lengths_of_next_stay

array([1., 3.])

In [92]:
lengths_of_next_stay = np.append(lengths_of_next_stay, np.nan)

In [131]:
df.loc[encounter_ids, ['readmitted']]['readmitted'].value_counts()['NO']

1

In [136]:
multiple_not_readmitted = 0
for patient in df['patient_nbr'].unique():
    encounter_ids = list(df[df['patient_nbr'] == patient]['encounter_id'])
    readmitted = df.loc[encounter_ids, ['readmitted']]['readmitted'].value_counts()
    if 'NO' in readmitted and readmitted['NO'] > 1:
        multiple_not_readmitted += 1

multiple_not_readmitted

44

In [144]:
df['len_next_stay'] = pd.Series()
for patient in df['patient_nbr'].unique():
    encounter_ids = list(df[df['patient_nbr'] == patient]['encounter_id'])
    readmitted = df.loc[encounter_ids, ['readmitted']]['readmitted'].value_counts()
    lengths_of_stay = df.loc[encounter_ids, 'time_in_hospital']
    df.loc[encounter_ids[:-1], ['len_next_stay']] = lengths_of_stay.values[1:]

df_readmitted_only = df.copy()
df_readmitted_only = df_readmitted_only[np.isfinite(df_readmitted_only['len_next_stay'])]
df_readmitted_only = df_readmitted_only[df_readmitted_only['readmitted'] != 'NO']

In [145]:
len(df_readmitted_only['patient_nbr'].unique())

16515

## Feature transformation

In [ ]:
columns = ["'{}'".format(i) for i in list(df.columns.values)]
','.join(columns)

In [149]:
# TODO masks vs sorted values; use the rest as numeric? 
masked_cat_features = ['max_glu_serum', 'A1Cresult']

num_to_cat_features = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'medical_specialty']

num_features = ['time_in_hospital', 'num_lab_procedures','num_procedures', 'num_medications', 
                         'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']

cat_features = ['readmitted', 'race', 'gender', 'change', 'diabetesMed', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 
                'glimepiride', 'acetohexamide', 'glipizide', 'glyburide','tolbutamide', 'pioglitazone', 'rosiglitazone', 
                'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin',
                'glipizide-metformin', 'glimepiride-pioglitazone','metformin-rosiglitazone', 'metformin-pioglitazone']

diag_features = ['diag_1', 'diag_2', 'diag_3']
age_features = 'age'

In [ ]:
df['total_bedrooms_mask'] = np.where(pandas.isnull(housing.total_bedrooms), 0, 1)
df['total_bedrooms'] = np.where(pandas.isnull(housing.total_bedrooms), 0, housing.total_bedrooms)

# ocean_proximity is a categorical, so replace it by one-hot-coded version
for (suffix,level) in [('med','<1H OCEAN'), ('far','INLAND'), ('isle','ISLAND'), ('bay','NEAR BAY'), ('near','NEAR OCEAN')]:
    assert np.sum(housing.ocean_proximity==level) > 0, f"No values for {level}"
    df[f'ocean_prox_{suffix}'] = np.where(housing.ocean_proximity==level, 1, 0)
del df['ocean_proximity']

# Scale the columns, in a sane way depending on what they mean
for col in ['longitude','latitude']:
    df[col] = (df[col] - np.min(df[col])) / (np.max(df[col]) - np.min(df[col]))
for col in ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']:
    df[col] = (df[col] - np.mean(df[col])) / np.std(df[col])
    
X = df.drop('median_house_value', axis=1).values
Y = df.median_house_value.values

print(X.shape, Y.shape)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_features)),
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

age_pipeline = Pipeline([
    ('selector', DataFrameSelector(age_features)),
    ('ordinal_encoder', OrdinalEncoder(categories=[['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)',
                                                    '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)']])),
    ('std_scaler', StandardScaler()),
])

class DiagEncoder(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = OneHotEncoder(*args, **kwargs, categories='auto', sparse=False)
    def fit(self, X, y=0):
        X = [[self.get_type(icd_str) for icd_str in x] for x in X]
        self.encoder.fit(X)
        return self
    def transform(self, X, y=0):
        X = [[self.get_type(icd_str) for icd_str in x] for x in X]
        return self.encoder.transform(X)
    def get_type(self, icd_str):
        if isinstance(icd_str, float) and math.isnan(icd_str):
            return('missing')
        elif icd_str.isnumeric():
            icd = int(icd_str)
        elif icd_str[:3].isnumeric():
            icd = int(icd_str[:3])
        else:
            return 'other'

        if (icd >= 390 and icd <= 459 or icd == 785):
            return 'circulatory'
        elif (icd >= 520 and icd <= 579 or icd == 787):
            return 'digestive'
        elif (icd >= 580 and icd <= 629 or icd == 788):
            return 'genitourinary'
        elif (icd == 250):
            return 'diabetes'
        elif (icd >= 800 and icd <= 999):
            return 'injury'
        elif (icd >= 710 and icd <= 739):
            return 'musculoskeletal'
        elif (icd >= 140 and icd <= 239):
            return 'neoplasms'
        elif (icd >= 460 and icd <= 519 or icd == 786):
            return 'respiratory'
        else:
            return 'other'

diag_pipeline = Pipeline([
    ('selector', DataFrameSelector(diag_features)),
    ('diag_encoder', DiagEncoder()),
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_to_cat_features + cat_features + masked_cat_features)),
    ('imputer', SimpleImputer(strategy='constant')),
    ('encoder', OneHotEncoder(categories='auto', sparse=False))

])

full_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('age_pipeline', age_pipeline),
    ('diag_pipeline', diag_pipeline),
    ('cat_pipeline', cat_pipeline),
])

X_train = full_pipeline.fit_transform(X_train)